<a href="https://colab.research.google.com/github/Bara7-Qusai/LangChaine/blob/main/RAG_Pipeline_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_community pypdf fastembed chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.0 MB/s eta 0:00:00

In [2]:
# get the huggingface token
import os
from getpass import getpass
from google.colab import userdata

HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('hugging_face_space')

Huggingface Token : ··········


In [3]:
# import PDF reader
from langchain_community.document_loaders.pdf import PyPDFLoader

# load the document
# Source credits: https://ncert.nic.in/textbook/pdf/lekl101.pdf
loader = PyPDFLoader("/content/lekl101.pdf")
data = loader.load()

In [4]:
# import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap = 0,
)

chunks = text_splitter.split_documents(data)


In [5]:
len(chunks)

46

In [6]:
chunks[0]

Document(metadata={'source': '/content/lekl101.pdf', 'page': 0, 'page_label': '1'}, page_content='1/I SELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter, and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short')

In [7]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [8]:
from langchain.vectorstores import Chroma

# initialize the vector store (save to disk)
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

In [9]:
# Let's define the query, since we are going to use it multiple times.
query = "what is importance of I sell my dreams according to the author"

In [10]:
# retrieve from vector db (load from disk) with query
retrieved_docs = db.similarity_search(query)
print(retrieved_docs[0].page_content)


10/KALEIDOSCOPE
Language Work
A. Vocabulary
Look up the meanings of the following phrases under ‘dream’
and ‘sell’ in the dictionary
dream         sell
dream on sell-by date
dream something away selling-point
(not) dream of doing something sell-out
dream something up selling price
look like a dream seller’s market
B. Grammar: Emphasis
Read this sentence carefully
One morning  at nine o’clock, while we were having
breakfast on the terrace of the Havana Riviera Hotel


In [11]:
# initialize the retriever
retriever = db.as_retriever(
search_type="mmr", #similarity
search_kwargs={'k': 4}
)

In [12]:
# import the question-answering chain and Huggingface Hub LLM
from langchain.llms import HuggingFaceHub

# define the llm
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",
model_kwargs={
"temperature":0.1,
"max_new_tokens":512,
"return_full_text":False,
"repetition_penalty":1.1,
"top_p":0.9
})


<ipython-input-12-fc8caf5cafb9>:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""


In [14]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [15]:
chain = (
{"context": retriever, "query": RunnablePassthrough()}
| prompt
| llm
| output_parser
)


In [16]:
response = chain.invoke(query)
response

'Based on the context provided, there isn\'t a clear statement from the author about the importance of "selling one\'s dreams." The text mentions several phrases related to dreams and selling, but it doesn\'t discuss the significance of selling dreams specifically. Therefore, I cannot provide a definitive answer based on the context given.'

In [17]:
response3 = chain.invoke("for how many years does the Author refer the lady to leave right away and not come back to Vienna")
response3

'The text mentions that thirteen years had passed since their first meeting when the lady was told she could go back to Vienna (Document 1). Therefore, the author refers to thirteen years as the time the lady should stay away from Vienna.'